# Medium

## Unique Paths

* https://leetcode.com/problems/unique-paths/description/
***
* Time Complexity:
    - naive: O(2$^{m * n}$)
        * for every cell, you have 2 options: you either go down or you go to the right
        * there are m * n cells and 2 options for each cell
    - topdown memo: O(m x n)
        * unlike the naive, you aren't repeating any work and return immediately so you only visit each cell once
    - bottom up: O(m x n)
        * you have 2 loops going from 1...m and 1...n
* Space Complexity:
    - naive: O(m + n)
        * you will have at most O(m + n) functions in the stack b/c you can go all the way down to the m-th row and go all the way to the right to the n-th column
    - topdown memo: O(m x n)
        * requires space for the 2D dp table
    - bottom up: O(m x n)
        * requires space for the 2D dp table
***
* naive:
    - basically dfs/backtracking
    - you either go down or you go right at each cell
    - if you reach the bottom-right where the finish line is, then increment the numWays
    - else, the number of ways to reach that cell is equal to the number of ways to reach the cell above and to the right of it
        * numWays[r][c] = numWays[r + 1][c] + numWays[r][c + 1]
* topdown memo:
    - during the dfs/backtracking, we can actually reach the same cells we've already traversed on so we're just repeating work
    - thus we can use a dp table to keep track of which cells we already have numWays on
    - if we reach a cell that we've seen, we just return the numWays without traversing it again
* bottom up:
    - the concept is the same as topdown memo but we realize a couple of things:
        1. we know that all the cells in the top rown, at row 0, are going to have 1 as their numWays
            * reason being, you cannot reach them unless you keep going right
        2. the same is true for all the cells in the first column, at col 0
            * you cannot reach those cells unless you keep going down
    - we can use that to our advantage and ask ourselves
        * we know that the numWays[r][c] = numWays[r + 1][c] + numWays[r][c + 1]
        * but since we start from the beginning and work our way to the bottom-right then this must be reversed
        * so our __recurrence relation__ must be:
            - __dp[r][c] = dp[r - 1][c] + dp[r][c - 1]__
            - since we've already calculated the cells in the first row and first col, we can use those tabulated values already to calculate newer cells

In [ ]:
/**
 * @param {number} m
 * @param {number} n
 * @return {number}
 */

 // naive dfs/backtracking
var uniquePaths = function(m, n) {
    let numWays = 0;

    const traverse = (r, c) => {
        if (r >= m || c >= n) {
            return;
        }
        if ((r === m - 1) && (c === n - 1)) {
            numWays++;
        }

        // go down
        traverse(r + 1, c);

        // or go right
        traverse(r, c + 1);
    }

    traverse(0, 0);
    return numWays;
};

// naive dfs/backtracking
var uniquePaths = function(m, n) {
    const traverse = (r, c, numWays) => {
        if (r < 0 || c < 0) {
            return 0;
        }
        if (r === 0 && c === 0) {
            numWays++;
            return numWays;
        }

        return traverse(r - 1, c, numWays) + traverse(r, c - 1, numWays);
    }

    return traverse(m - 1, n - 1, 0);
};

// topdown memo
var uniquePaths = function(m, n) {
    const dp = Array.from({length: m}, () => Array.from({length: n}));

    const traverse = (r, c, numWays) => {
        if (r < 0 || c < 0) {
            return 0;
        }
        if (r === 0 && c === 0) {
            numWays++;
            return numWays;
        }
        if (dp[r][c] !== undefined) {
            return dp[r][c];
        }

        dp[r][c] = traverse(r - 1, c, numWays) + traverse(r, c - 1, numWays);
        return dp[r][c];
    }

    // console.log({dp})
    return traverse(m - 1, n - 1, 0);
};

// bottom-up dp
var uniquePaths = function(m, n) {
    const dp = Array.from({length: m}, () => Array.from({length: n}));

    for (let r = 0; r < m; r++) {
        dp[r][0] = 1;
    }

    for (let c = 0; c < n; c++) {
        dp[0][c] = 1;
    }

    for (let r = 1; r < m; r++) {
        for (let c = 1; c < n; c++) {
            dp[r][c] = dp[r - 1][c] + dp[r][c - 1];
        }
    }

    return dp[m - 1][n - 1];
}


## Longest Common Subsequence

* https://leetcode.com/problems/longest-common-subsequence/description/
***
* Time Complexity:
    - naive: O(2$^{n}$)
        * if text1[i] !== text2[j], then we have 2 choices to traverse: [i + 1, j] or [i, j + 1]
            - essentially, we disregard text1[i] and instead look at subsequence from text1[i + 1...n]
            - or disregard text2[j] and look at subsequence from text2[j + 1...n]
    - topdown memo: O(m x n), where m = length of text1 and n = length of text2
        * by using a dp table, we immediately solve any overlapping subproblems
        * thus we only need to solve for all subproblems of [i,j]
            - and since i = m and j = n, we have to solve m x n total problems
    - bottom up: O(m x n)
        * similar to the topdown memo
        * we fill out a table and solve m x n total problems to get the answer
* Space Complexity:
    - naive: O(max(m, n))
        * uses recursion so we need space for the function stack
        * when text1[i] !== text2[j] we have to traverse on [i + 1, j] and [i, j + 1]
            - i is bounded by the length of text1 and j is bounded by the length of text2 since we return immediately when i and j are equal or greater than m and n respectively
            - therefore, the max height of our recursion will be the max length between text1 and text2
    - topdown memo: O(m x n)
        * requires space for the dp table which is O(m x n)
    - bottom up: O(m x n)
        * requires space for the dp table which is O(m x n)
***
* I figured out the bottom up dp algorithm first before the naive/top down so i'll start with that
* bottom up dp:
    - visualize filling out a table where the rows represent the subsequences of text1 and the columns are subsequences of text2
    - when we fill out the first row and the first column respectively, what are we actually doing?
        * for the first row, we are checking if text1[0] is a subsequence of text2[0 ... n]
        * for the first col, we are checking if text2[0] is a subsequence of text1[0...n]
        * if at any point the first char of either of these texts are a subsequence of a section of the other text, then the rest of this text also contains that subsequence:
            - e.g. text1 = 'bac', text2 = 'a'
            - our first row would look like: [0, 1, 1]
                * 'b' !== 'a'
                * 'a' === 'a'
                * 'c' !== 'a' BUT we know that the previous subsequence DOES contain at least 1 char so we include it even though the chars don't match
    - once we fill out the first row and first col, we can then look at the __recurrence relation__:
        * __dp[i][j] =__ 
            - __1 + dp[i - 1][j - 1], if text1[i] === text2[j]__
            - __Math.max(dp[i - 1][j], dp[i, j - 1]), if text1[i] !== text2[j]__
        * essentially, if the current strings match at i and j, then look at the previous subsequences of both and add 1 to them
            - thus we look at dp[i - 1][j - 1]
        * if they don't match, then we look at previous subsequences of either text
            - dp[i - 1][j] = look at previous subsequences of text1
            - dp[i][j - 1] = look at previous subsequences of text2
* naive algorithm:
    - essentially use the same recurrence relation but done recursively
* dp algorithm:
    - notice that we might be solving for the same [i,j] subsequences so we use a dp table to keep track of those values
    - if we have already solved for it, then just return it

In [1]:
/**
 * @param {string} text1
 * @param {string} text2
 * @return {number}
 */

 // naive algorithm
 // Time: O(2^n)
 // Space: O(max(m, n))
var longestCommonSubsequence = function(text1, text2) {
    if (text1 === text2) return text1.length;

    const traverse = (i, j) => {
        if (i >= text1.length || j >= text2.length) return 0;

        if (text1[i] === text2[j]) {
            return 1 + traverse(i + 1, j + 1);
        }
        else {
            return Math.max(
                traverse(i + 1, j),
                traverse(i, j + 1)
            );
        }
    }

    return traverse(0, 0);
};


// topdown memo
// Time: O(m x n)
// Space: O(m x n)
var longestCommonSubsequence = function(text1, text2) {
    if (text1 === text2) return text1.length;
    const m = text1.length;
    const n = text2.length;
    const dp = Array.from({length: m}, () => []);

    const traverse = (i, j) => {
        if (i >= m || j >= n) return 0;
        if (dp[i][j] !== undefined) {
            return dp[i][j];
        }

        if (text1[i] === text2[j]) {
            dp[i][j] = 1 + traverse(i + 1, j + 1);
        }
        else {
            dp[i][j] = Math.max(
                traverse(i + 1, j),
                traverse(i, j + 1)
            );
        }

        return dp[i][j];
    }

    return traverse(0, 0);
};

// bottom up
// Time: O(m x n)
// Space: O(m x n)
var longestCommonSubsequence = function(text1, text2) {
    if (text1 === text2) return text1.length;

    // create the dp table
    const m = text1.length;
    const n = text2.length;
    const dp = Array.from({length: m}, () => []);

    // fill out the first row and first col
    dp[0][0] = text1[0] === text2[0] ? 1 : 0;

    // row
    for (let i = 1; i < m; i++) {
        dp[i][0] = text1[i] === text2[0] ? 1 : dp[i - 1][0];
    }

    // col
    for (let j = 1; j < n; j++) {
        dp[0][j] = text1[0] === text2[j] ? 1 : dp[0][j - 1];
    }

    // looping through the entire table
    for (let i = 1; i < m; i++) {
        for (let j = 1; j < n; j++) {
            if (text1[i] === text2[j]) {
                dp[i][j] = 1 + dp[i - 1][j - 1];
            }
            else {
                dp[i][j] = Math.max(dp[i - 1][j], dp[i][j - 1]);
            }
        }
    }

    return dp[m - 1][n - 1];
}

## Best Time to Buy and Sell Stock with Cooldown

* https://leetcode.com/problems/best-time-to-buy-and-sell-stock-with-cooldown/description/
***
* Time Complexity: O(n)
    - basically we just traverse from [0...n]
    - our dp table helps us to maintain O(n) complexity b/c of the key
        * normally it would be around O(2$^{n}$) b/c for each index from 0 to n, we can make 2 decisions:
            - if we haven't bought, we can buy or cooldown
            - if we have bought, we can sell or cooldown
            - if we have already sold, we must cooldown but this case is taken care of by skipping over the next index and going to index + 2
* Space Complexity: O(n)
    - needs space for the dp table
***
* __FOR THESE STOCK TYPE QUESTIONS WE MUST BE AWARE OF THREE STATES: BUY, SELL, HOLD__
* if we draw out a decision tree, each node in the tree represents an index and we have several options
    - i = 0: we can either buy or hold
        * we can't sell b/c we haven't bought anything
        * and we can ALWAYS HOLD (just do nothing)
    - i = 1:
        * if we bought at i = 0, we can either sell or cooldown
        * if we hold at i = 0, we can either buy or cooldown
    - i = 2:
        * if we hold at i = 1, we can either buy or cooldown
        * if we bought at i = 1, we can either sell or cooldown
        * if we sold at i = 1, WE MUST COOLDOWN which means this index would've been skipped and we would've arrived at i = 3 if this was the case
* thus our __recurrence relation is so:__
    * __dp[i][canBuy] = Math.max(dp[i + 1][!canBuy], dp[i + 1][canBuy]), if canBuy = true__
        - basically if we are buying, then we have to look at the max ofif we sell or hold on the next stock
    * __dp[i][canBuy] = Math.max(dp[i + 2][!canBuy], dp[i + 1][canBuy]), if canBuy = false__
        - basically if we are selling, we HAVE to take a cooldown and look at the max profit starting at i + 2 and if we decided to cooldown and not sell at i

In [2]:
/**
 * @param {number[]} prices
 * @return {number}
 */

 // Time Complexity: O(n)
 // Space Complexity: O(n)
var maxProfit = function(prices) {
    if (prices.length === 1) return 0;

    // key = 'index-boolean', val = max profit
    // e.g. 0-true, 10
    const dp = new Map();

    const traverse = (i, canBuy) => {
        if (i >= prices.length) return 0;
        
        const key = `${i}${canBuy}`
        if (dp.has(key)) {
            return dp.get(key);
        }

        // for our stock we can either buy, sell, or hold (cooldown)
        // this basically represents branches in our decision tree
        // however, we know that the problem has 1 constraint:
        // you MUST cooldown after you sell for at least 1 day (or 1 index)
        // so in order to accommodate for this, we must keep track of this state somehow

        // regardless of if we've bought or sold before
        // we can ALWAYS choose the option to cooldown or hold
        let cooldown = traverse(i + 1, canBuy);
        
        // if we are allowed to buy
        if (canBuy) {
            // we subtract prices[i] b/c we are LOSING profit by buying the stock
            // we traverse to the next index and we set the boolean to false
            // b/c we've already bought and do NOT want to buy again
            let buy = traverse(i + 1, false) - prices[i];
            dp.set(key, Math.max(buy, cooldown));
        }
        else {
            // since we are selling the stock, the next state MUST be to cooldown
            // so that's why we move 2 indices away
            // and we add prices[i] b/c we MADE profit (maybe) by selling the stock
            // we also set buying to true here b/c we've already sold
            // we're allowed to do it after we've cooldowned
            let sell = traverse(i + 2, true) + prices[i];
            dp.set(key, Math.max(sell, cooldown));
        }

        return dp.get(key);
    }

    return traverse(0, true);
};

## Coin Change II

* https://leetcode.com/problems/coin-change-ii/description/
***
* Time Complexity:
    - naive: O(2$^{amount}$)
        * you have 2 options for every coin
            - don't include the coin in the combination to make the amount
            - include the coin to make the amount
            - and the height of the tree = amount b/c if you have coins = [1] and amount = 10, you will only have 1 branch that will go down all the way to the amount
    - topdown memo: O(n * amount), where n = # of coins
        * you essentially fill up your dp table but recursively
        * and your dp table has O(n * amount) subproblems
    - bottom-up dp: O(n * amount)
        * also filling up a dp table but iteratively and there are O(n * amount) subproblems
* Space Complexity:
    - naive: O(amount)
        * using recursion so you have to have space for the functions on the stack
        * and the amount of functions on the stack is equal to the height of the tree which is O(amount)
    - topdown memo: O(n * amount)
        * requires space for your dp table
        * also uses recursion but that is not the dominant term: O(amount)
    - bottom-up dp: O(n * amount)
        * requires space for your dp table
    - bottom-up dp - space optimized: O(amount)
        * the bottom-up solution only requires the current and previous rows which can just be squashed into one row
        * so when we only use 1 row, our array is still the length of the amount + 1
        * therefore, our space is O(amount)
***
* naive:
    - we know that if the amount = 0, there is 1 COMBINATION that can make it, which is the __empty set__
    - and similar to other backtracking/dfs problems we have TWO options:
        * we can EXCLUDE the current coin and proceed to other coins after it with the same current amount
        * we can INCLUDE the current coin and proceed to other coins after it
    - in addition, the ORDER of the coins in the combination DOES NOT MATTER
        * so if coins = [1,2] and amount = 4
            - [1,1,2] = [1,2,1] = [2,1,1] even though the order is different
        * __to solve this, we can pass in the INDEX so that if we have reached that coin's index, we only need to look at coins with an index AFTER that one and we don't backtrack on those__
* topdown memo:
    - we are actually repeating some subproblems so we can use a dp table to keep track of those and return immediately
    - our dp table = dp[index][amount]
        * dp[i][j] represents the amount of combinations that make the amount, j, using the first i coins
* bottom-up dp:
    - we have to be wary of a special case to initialize our dp table for bottom-up
        * dp[0][0] = 1
            - __if amount = 0, we can use the EMPTY SET__ to create this
        * this also helps us to initialize this further by suggesting that dp[i][0] = 1 as well since the empty set will ALWAYS be a valid combination to make amount 0 regardless of how many coins we use
        * and this also helps with initializing the first row as well since we know that dp[0][j] = 0 for j >= 1
            - this is b/c we cannot make any amount > 0 with 0 coins
    - once we've initialized, our __recurrence relation__ is:
        * __dp[i][j] = dp[i - 1][j] + dp[i][j - coins[i - 1]]__
            - dp[i - 1][j] = exclude the coin and only use the previous combination of coins
            - dp[i][j - coins[i - 1]] = include the coin to create the amount
                * __THE REASON WHY WE USE coins[i - 1] INSTEAD OF coins[i] IS B/C OF THE SPECIAL CASE OF THE EMPTY SET__
                    - when i = 0, that represents using 0 coins to make the amount
                    - when i = 1, that represents using the first coin to make the amount BUT THE FIRST COIN IS AT INDEX 0 IN COINS
                        * so to account for this, we use coins[i - 1]
                        * therefore, i = 1, coins[1 - 1] = coins[0]
* bottom-up dp - space optimized:
    - notice how our recurrence relation only uses the current and previous rows
    - thus we can actually squish the entire table into just 1 row
    - by doing this, our __optimized recurrence relation__ changes
        * __dp[amount] = dp[amount] + dp[amount - coin]__
            - since we only use 1 row for the entire dp algorithm, we can drop 1 dimension from our dp which just leaves us with the amount, j
    - dropping the row dimension also allows us to change the outer loop to instead loop using values instead of the i index
        * this bypasses the need for index compensation from the non-space optimized solution and simplifies the dp

In [2]:
/**
 * @param {number} amount
 * @param {number[]} coins
 * @return {number}
 */

// naive
var change = function(amount, coins) {
    // there's only 1 COMBINATION to make 0
    // it's basically the empty set
    if (amount === 0) return 1;

    const traverse = (index, curAmount) => {
        if (curAmount > amount) return 0;
        if (curAmount === amount) return 1;
        if (index >= coins.length) return 0;

        // index, curAmount + coins[index] => use the same coin and update the amount
        // index + 1, curAmount => use a different coin but keep the same amount
        return traverse(index, curAmount + coins[index]) + traverse(index + 1, curAmount); 
    }

    return traverse(0, 0)
};

// topdown memo
var change = function(amount, coins) {
    if (amount === 0) return 1;
    const dp = Array.from({length: coins.length}, () => []);

    const traverse = (index, curAmount) => {
        if (curAmount > amount) return 0;
        if (curAmount === amount) return 1;
        if (index >= coins.length) return 0;

        if (dp[index][curAmount]) {
            return dp[index][curAmount];
        }

        let ways = 0;
        
        // use the current coin until we reach the amount of over the amount
        ways += traverse(index, curAmount + coins[index]);

        // or we disregard current coin and move on to the ones after it
        // by doing this, we also disregard dupes b/c we do not reuse any coins we've already used
        ways += traverse(index + 1, curAmount);

        dp[index][curAmount] = ways;

        return dp[index][curAmount];
    }

    return traverse(0, 0);
}

// bottom-up dp
var change = function(amount, coins) {
    if (amount === 0) return 1;

    // dp[0...coins.length][0...amount]
    const dp = Array.from({length: coins.length + 1}, () => []);

    // THIS IS A SPECIAL CASE
    // WE CAN MAKE AMOUNT = 0 WHEN WE USE 0 COINS!!!
    // remember that the Coin Change II problem asks for the NUMBER OF COMBINATIONS
    // the EMPTY SET is a valid combination
    dp[0][0] = 1;

    for (let i = 1; i <= coins.length; i++) {
        // we always have 1 combination that can make amount 0
        // which is the EMPTY SET
        dp[i][0] = 1;

        for (let j = 1; j <= amount; j++) {
            // the top row is initialized to 0 b/c there is no way to make any amount over 0
            // using 0 coins
            dp[0][j] = 0;

            // we have the option of disregarding the current coin in the combination
            // and only using combinations of the previous coins
            dp[i][j] = dp[i - 1][j];

            // the other option is to include the coin in the combination
            // but we must determine if adding it will not be over the amount

            // THE REASON WHY WE USE coins[i - 1] INSTEAD OF coins[i] IS BECAUSE OF THE SPECIAL CASE
            // WHERE WE USE THE EMPTY SET (0 COINS)
            // SO WHEN WE HAVE i = 0, THAT IS ACTUALLY NOT THE FIRST COIN
            // THAT REPRESENTS WHEN WE USE 0 COINS
            // i = 1 = when we use the FIRST COIN in coins even though i != 0
            // therefore, we use coins[i - 1]. so if i = 1, then coins[1 - 1] = coins[0]
            if (j - coins[i - 1] >= 0) {
                dp[i][j] += dp[i][j - coins[i - 1]];
            }
        }
    }

    return dp[coins.length][amount];
}

// bottom-up dp space optimized
var change = function(amount, coins) {
    if (amount === 0) return 1;

    // in the original bottom-up algorithm, we only ever needed the current and previous rows
    // and not the entire table
    // therefore, we can squeeze this into just 1 array
    const dp = Array.from({length: amount + 1}, () => 0);
    dp[0] = 1;

    // the loop structure is very similar to Coin Change
    for (let coin of coins) {
        for (let amt = coin; amt <= amount; amt++) {
            // this is basically the same as the previous recurrence relation but simplified
            // it is essentially: dp[amt] = dp[amt] + dp[amt - coin];
            // so we either disregard the current coin in the amt
            // OR we use it

            // and since we only use a 1D array here instead of a 2D, we can drop the row index
            // and only use the amount index
            // this is also why we do not need an i index for the outer loop and only grab the coin values

            // and we do not need to check for any conditionals where it is out of bounds b/c
            // we start off at amt = coin
            // and if amt = coin, then amt - coin = 0
            dp[amt] += dp[amt - coin];
        }
    }

    return dp[amount];
}

## Target Sum

* https://leetcode.com/problems/target-sum/description/
***
* Time Complexity:
    - naive: O(2$^{n}$), where n = length of nums
        * basically backtracking/dfs
        * at every node, you have 2 options:
            - either add the number
            - or you subtract the number
        * and since you have to do this for every number in nums, the height of the tree is n
    - topdown memo: O(n * target)
        * you're essentially filling out a dp table
        * and the dp table has O(n * target) subproblems
    - bottom-up: O(n * target)
        * also filling out a dp table and there are O(n * target) subproblems
* Space Complexity:
    - naive: O(n)
        * uses recursion so requires space for the function stack
        * there will be O(n) functions in the stack b/c the deepest a branch will go is equal to length of nums which is n
    - topdown memo: O(n * target)
        * requires space for the dp table
        * also uses recursion
    - bottom-up: O(target)
        * dp table is equal to (target + sum + 1) * 2
        * if target ~= sum, then this is basically target * 2 or O(target)
***
* naive:
    - basically have 2 options:
        * you add the current number
        * or subtract the current number
    - you do this until you've accounted for all the numbers in nums
* topdown memo:
    - you're actually doing duplicate work here so you can use a dp table to memoize some subproblems
    - your dp table would then return if dp[index][sum] is already solved
* bottom-up:
    - the original problem can be summarized like so:
        * find a subset of nums that need to be positive and the rest of them negative, such that the sum is equal to the target
        * essentially: sum(Positve) - sum(Negative) = target
    - we know that the Partition Equal Subset Sum's goal is thus: find a subset = target / 2
        * using this goal, we can transform our equation to match this:
            - sum(P) - sum(N) = target
            - add sum(P) + sum(N) to both sides 
            - => 2 * sum(P) = target + sum(P) + sum(N)
                * reason why we add sum(N) is b/c we want to just isolate sum(P) on the left side and get rid of sum(N)
            - we can condense the right side => 2 * sum(P) = target + sum(nums)
            - => __sum(P) = (target + sum(nums)) / 2__
    - looking at this equation, we can also use this to check if we can get a solution with nums that will equal to target
        * we know that sum(P) HAS to be an even number
    - using that equation, we then create a dp table equal to that sum and initialize dp[0] = 1
        * there will always exist a subset that will create a sum of 0, the empty set
    - then we loop for every coin and for each coin, we loop from the sum...coin
        * 2 reasons why we do this:
            - this is a 0/1 knapsack, NOT UNBOUNDED KNAPSACK
            - so to avoid REUSING the same coins infinitely, we count backwards from the sum since we haven't used those earlier sums yet.
                * they've all been initialized to 0
            - the second reason is that if we go from sum...coin instead of 0, we do not have to check the condition (num <= sum) b/c if sum = coin, then sum - coin = 0.

In [4]:
/**
 * @param {number[]} nums
 * @param {number} target
 * @return {number}
 */

// naive
var findTargetSumWays = function(nums, target) {
    const traverse = (i, sum) => {
        if (i >= nums.length) {
            return sum === target ? 1 : 0;
        }

        return traverse(i + 1, sum + nums[i]) + traverse(i + 1, sum - nums[i]);
    }

    return traverse(0, 0);
};

// topdown memo
var findTargetSumWays = function(nums, target) {
    // dp[0...nums.length][0...target]
    // the reason why we use a map is b/c we have to account for NEGATIVE targets
    const dp = Array.from({length: nums.length}, () => new Map());

    const traverse = (i, sum) => {
        if (i >= nums.length) {
            return sum === target ? 1 : 0;
        }

        if (dp[i].has(sum)) {
            return dp[i].get(sum);
        }
        
        dp[i].set(sum, traverse(i + 1, sum + nums[i]) + traverse(i + 1, sum - nums[i]));
        return dp[i].get(sum);
    }

    return traverse(0, 0);
};

// bottom-up dp
var findTargetSumWays = function(nums, target) {
    let sum = 0;
    nums.forEach(num => sum += num);

    // if you can't even add all the numbers up to the target, then it can't be
    // also, similar to the Partition Equal Subset Sum question, the sum of the
    // positive (+) integers MUST be even
    if (sum < Math.abs(target) || (sum + target) % 2 === 1) {
        return 0;
    }

    // this is all essentially the same as Partition Equal Subset Sum
    sum = Math.trunc((target + sum) / 2);
    const dp = Array.from({length: sum + 1}, () => 0);
    dp[0] = 1;

    for (let num of nums) {
        // if we go from sum ... num, then we do not need to check if the condition
        // (num <= sum) is true so that we do not check for any index < 0
        for (let i = sum; i >= num; i--) {
                dp[i] += dp[i - num];
        }
    }

    return dp[sum];
}

## Interleaving String

* https://leetcode.com/problems/interleaving-string/description/
***
* Time Complexity:
    - naive: O(2$^{m + n}$), where n = size of s1 and m = size of s2
        * you have 2 options per node
            - either use s1
            - or use s2
        * the recursion goes as deep as the length of s3, which is m + n
    - topdown: O(m * n)
        - essentially filling out a dp table the size of m * n
    - bottom-up: O(m * n)
        - filling out a dp table the size of m * n
* Space Complexity:
    - naive: O(m + n)
        * uses recursion so needs space for the functions in the stack
        * the recursion goes as deep as the length of s3, which is m + n
    - topdown: O(m * n)
        * needs space for a dp table which is m * n
    - bottom-up: O(m * n)
        * needs space for a dp table which is m * n
    - bottom-up space optimized: O(m)
        * only use the current and previous rows, dp[i] and dp[i - 1], respectively so we can condense the table down to just one row
        * the length of the row is equal to s2 which is m
***
* naive:
    - the interleaving part is confusing but essentially you are seeing if you can weave s1 and s2 to make s3
        * the condition of |n - m| <= 1 is always going to be true
        * take for example s1 = "ab", s2 = "abad", s3 = "aabadb"
            - this is made by splitting s1 = "a" + "b" and putting s2 in the middle of that, so "a"(s1) + "abad"(s2) + "b"(s1)
            - "a" is the first substring of s1 and "abad" is the first substring of s2, which are s1<sub>0</sub> and s2<sub>0</sub> respectively
            - but if you decided to use the entirety of s1 and then s2, then s1<sub>0</sub> = "ab"
            - in essence, __using s1 or s2 will either contribute to the current substring OR use a new substring__
                * if you use s1 and the previous interleaving ALSO used s1, then that contributes to the substring
                * but if you use s2 and the previous interleaving used s1, you are using a new substring of s2
                * so either way, the constraint will ALWAYS be true
    - so if we want to weave s1 and s2 together, we can do what we always do with dfs/backtracking
        * either start with s1
        * or start with s2
        * but we also need to make sure of 1 thing, DOES THE CHARACTER EVEN MATCH THE ONE IN S3
            - if s3[0] = "a" and s1[0] = "b", why would we continue traversing starting with s1?
            - every interleaving using s1[0] will ALWAYS be false
* topdown memo:
    - we are actually repeating work in the naive algorithm so we can keep track of combinations of i and j that have interleaved already
    - so our __recurrence relation__ is:
        * __dp[i][j] = (dp[i + 1][j] && s1[i] === s3[i + j - 1]) || (dp[i][j + 1] && s2[j] === s3[i + j - 1]), where [i + j - 1] is the current index of s3__
* bottom-up dp:
    - essentially doing what the topdown memo is doing but in reverse
    - in this algorith, 0 DOES NOT REPRESENT THE FIRST INDEX OF s1 or s2
        * 0 = THE EMPTY SET, using 0 characters from s1 or s2
    - dp[i][j] represents if we can interleave string s3[0...(i +j - 1)] using s1[0...i] and s2[0...j]
        * THEREFORE dp[0][0] = true
        * e.g. if s1 = "", s2 = "", and s3 = "" is TRUE
            - s1 = 0, s2 = 0, then s3 MUST be 0
    - so using this knowledge, we can create our dp table to be dp[n + 1][m + 1] in size, where n = s1.length and m = s2.length
        * why do we use n + 1 and m + 1?
        * b/c of the special case of dp[0][0]
            - since it represents using no characters from either string, then dp[1][1] is using 1 character from both s1 and s2
            - so we can think of it like a __SLICE__ operation
    - to initialize the table:
        * dp[0][0] = true
        * dp[i][0] = dp[i - 1][0] && s1[i - 1] === s3[i + j - 1] for 1...s1.length
            - if we only interleave with characters from s1
            - basically, if the current character at s1 matches the current character at s3, then can we interleave with the current character at s1 AND the previous substring of s1?
            - also we check s1[i - 1] B/C of the special case of dp[0][0]
            - s1[0] is not the first character of s1.
                * it represents the using no characters from s1
                * so if i = 1, then s1[1 - 1] = the first character of s1
        * dp[0][j] = dp[0][j - 1] && s2[j - 1] === s3[i + j - 1] for 1...s2.length
            - same thing as s1 but for s2
    - we then loop from 1...s1.length and 1...s2.length and check the __recurrence relation of the bottom-up dp:__
        * __dp[i][j] = (dp[i - 1][j] && s1[i - 1] === s3[i + j - 1]) || (dp[i][j - 1] && s2[j - 1] === s3[i + j - 1])__
* bottom-up space optimized:
    - notice in the recurrence relation that we only ever use the current and previous rows, dp[i] and dp[i - 1]
    - thus we can compress the entire table down into just 1 row and can remove AN ENTIRE DIMENSION from our recurrence relation
    - __space optimized recurrence relation__:
        * __dp[j] = (dp[j] && s1[i - 1] === s3[i + j - 1]) || (dp[j - 1] && s2[j - 1] === s3[i + j - 1])__
        * we only remove the row dimension from our dp calls, i, and we keep the rest of the conditional intact since we still need to check if the characters of s1/s2 match with s3

In [2]:
/**
 * @param {string} s1
 * @param {string} s2
 * @param {string} s3
 * @return {boolean}
 */

 // naive algorithm
var isInterleave = function(s1, s2, s3) {
    const n = s1.length;
    const m = s2.length;

    if (n === 0 || m === 0) {
        return s1 === s3 || s2 === s3;
    }

    const traverse = (i, j, k, str) => {
        // console.log({i, j, k, str, isEqual: str === s3})
        if (str.length === s3.length && (i === n && j === m)) return str === s3;
        
        let canInterleave = false;
        
        // start with s1
        if (i < n && s1[i] === s3[k]) canInterleave ||= traverse(i + 1, j, k + 1,str + s1[i]);

        // start with s2
        if (j < m && s2[j] === s3[k]) canInterleave ||= traverse(i, j + 1, k + 1, str + s2[j]);

        return canInterleave;
    }

    return traverse(0, 0, 0, "");
};


// topdown memo
var isInterleave = function(s1, s2, s3) {
    const n = s1.length;
    const m = s2.length;

    if (n === 0 || m === 0) {
        return s1 === s3 || s2 === s3;
    }

    const dp = Array.from({length: n + 1}, () => []);

    const traverse = (i, j, k, str) => {
        if (str.length === s3.length && (i === n && j === m)) return str === s3;

        if (dp[i][j] !== undefined) {
            return dp[i][j];
        }
        
        let canInterleave = false;
        
        // start with s1
        if (i < n && s1[i] === s3[k]) canInterleave ||= traverse(i + 1, j, k + 1,str + s1[i]);

        // start with s2
        if (j < m && s2[j] === s3[k]) canInterleave ||= traverse(i, j + 1, k + 1, str + s2[j]);

        dp[i][j] = canInterleave;

        return dp[i][j];
    }

    return traverse(0, 0, 0, "");
};

// bottom-up dp
var isInterleave = function(s1, s2, s3) {
    const n = s1.length;
    const m = s2.length;

    // must interleave the entirety of s1 and s2
    // so if their lengths don't even match s3, then it cannot be interleaved
    if (n + m < s3.length) return false;

    // in the case where s1 = "abc", s2 = "", s3 = "abc"
    if (n === 0 || m === 0) {
        return s1 === s3 || s2 === s3;
    }

    // dp[0][0] index represents using NO characters
    // NOT taking the first indices from each string
    const dp = Array.from({length: n + 1}, () => []);

    // we use no characters from either s1 or s2
    dp[0][0] = true;

    // since 0 represents using no characters
    // s1[0] !== first character in s1
    // s1[i - 1] = first character
    for (let i = 1; i <= n; i++) {
        dp[i][0] = dp[i - 1][0] && (s1[i - 1] === s3[i - 1]);
    }

    for (let j = 1; j <= m; j++) {
        dp[0][j] = dp[0][j - 1] && (s2[j - 1] === s3[j - 1]);
    }

    for (let i = 1; i <= n; i++) {
        for (let j = 1; j <= m; j++) {
            dp[i][j] = false;

            // start with s1
            dp[i][j] ||= (dp[i - 1][j] && s1[i - 1] === s3[i + j - 1]);

            // start with s2
            dp[i][j] ||= (dp[i][j - 1] && s2[j - 1] === s3[i + j - 1]);
        }
    }

    return dp[n][m];
}

// bottom-up dp - space optimized
var isInterleave = function(s1, s2, s3) {
    const n = s1.length;
    const m = s2.length;

    // must interleave the entirety of s1 and s2
    // so if their lengths don't even match s3, then it cannot be interleaved
    if (n + m < s3.length) return false;

    // in the case where s1 = "abc", s2 = "", s3 = "abc"
    if (n === 0 || m === 0) {
        return s1 === s3 || s2 === s3;
    }

    // dp[0][0] index represents using NO characters
    // NOT taking the first indices from each string

    // notice from earlier that we only ever use the current row or the previous row
    // so basically only i and i - 1
    // therefore, we can reduce the space down to just 1 row
    const dp = Array.from({length: m + 1});

    // by removing 1 row, we can remove a dimension ENTIRELY
    // so dp[i][j] can be condensed down to dp[j]
    for (let i = 0; i <= n; i++) {
        for (let j = 0; j <= m; j++) {
            if (i === 0 && j === 0) {
                dp[j] = true;
            }
            else if (i === 0) {
                dp[j] = dp[j - 1] && s2[j - 1] === s3[i + j - 1];
            }
            else if (j === 0) {
                dp[j] = dp[j] && s1[i - 1] === s3[i + j - 1];
            }
            else {
                dp[j] = (dp[j - 1] && s2[j - 1] === s3[i + j - 1]) || (dp[j] && s1[i - 1] === s3[i + j - 1]);
            }
        }
    }

    return dp[m];
}

## Unique Paths II

* https://leetcode.com/problems/unique-paths-ii/description/
***
* Time Complexity:
    - naive: O(2$^{m x n}$)
        * for every cell that you encounter, you have 2 options:
            1. go down [r + 1][c]
            2. or go right [r][c + 1]
        * this naive algorithm will encounter all cells
    - topdown memo: O(m x n)
        * the algorithm will still either go down or right but now it will return from any cells it has already encountered
        * by not repeating this work, you will effectively visit every cell but now going down or right will not occur unless it is a new cell
        * this reduces the time complexity significantly
    - bottom-up dp: O(m x n)
        * filling out the dp table will have us visit each cell in obstacleGrid
* Space Complexity:
    - naive: O(m + n)
        * uses recursion so requires some space for the function calls in the stack
        * the number of function calls is bounded by the numRows + numCols in obstacleGrid
            - reason being, if you look at the recursion, you'll notice that it will recurse all the way down the grid first (numRows), then it would recurse all the way to the right (numCols)
            - and once it reaches obstacleGrid[numRows - 1][numCols - 1] it will return
            - so this is essentially numRows + numCols
    - topdown memo: O(m x n)
        * requires space for recursion, O(m + n)
        * requires space for the memo table which is basically for each cell in obstacleGrid
    - bottom-up dp: O(m x n)
        * requires space for the dp table
***
* naive:
    - base case:
        * we know that if we reach an obstacle or we go out of bounds (OoB), then we have to return 0 b/c there are 0 ways to reach that cell
        * and if we reach obstacleGrid[numRows - 1][numCols - 1] we return 1
            - however, this condition occurs AFTER checking if obstacleGrid[numRows - 1][numCols - 1] === 1
            - this is b/c if the last cell in obstacleGrid is an obstacle itself, then there is no way to reach it
    - after we go through those conditions, we traverse to the right and left
* topdown memo:
    - our naive algorithm does not account for cells that we have encountered already so it will duplicate work
    - in order to avoid this, we create a memo table and return if we have already seen it
    - memo[r][c] represents the # of ways we reach last cell of obstacleGrid from [r, c]
* bottom-up dp:
    - looking at what we've done for the naive and memoized algorithms, we can determine our __recurrence relation__ to be:
        * __dp[r][c] = dp[r - 1][c] + dp[r][c - 1]__
        * dp[r][c] represents the # of ways to get to obstacleGrid[r][c] b/c we want to eventually return dp[numRows - 1][numCols - 1]
        * therefore, our base case would be dp[0][0] = 0 or 1 depending on if obstacleGrid[r][c] is an obstacle itself
        * we can then use dp[0][0] as a starting point to fill out row 0 and col 0
            - for row, dp[r][0] = dp[r - 1][0] or 0 if obstacleGrid[r][0] is an obstacle
            - for col, dp[0][c] = dp[0][c - 1] or 0 if obstacleGrid[0][c] is an obstacle
        * this is the reason why our recurrence relation is dp[r - 1][c] and dp[r][c - 1] instead of dp[r + 1][c] or dp[r][c + 1] 
            - since we start from dp[0][0] and we loop from [1...numRows] and [1...numCols] we have not even solved for dp[r + 1][c] or dp[r][c + 1] yet
            - we must therefore look at previous dp values to solve new ones

In [1]:
/**
 * @param {number[][]} obstacleGrid
 * @return {number}
 */

 // naive
var uniquePathsWithObstacles = function(obstacleGrid) {
    const numRows = obstacleGrid.length;
    const numCols = obstacleGrid[0].length;

    const traverse = (r, c) => {
        // OoB or in obstacle
        if ((r < 0 || r >= numRows) ||
            (c < 0 || c >= numCols) ||
            (obstacleGrid[r][c] === 1)) {
                return 0;
        }

        if (r === numRows - 1 && c === numCols - 1) {
            return 1;
        }

        return traverse(r + 1, c) + traverse(r, c + 1); 
    }

    return traverse(0, 0);
};

// topdown memo
var uniquePathsWithObstacles = function(obstacleGrid) {
    const numRows = obstacleGrid.length;
    const numCols = obstacleGrid[0].length;
    const memo = Array.from({length: numRows}, () => []);

    const traverse = (r, c) => {
        // OoB or an obstacle
        if ((r < 0 || r >= numRows) ||
            (c < 0 || c >= numCols) ||
            (obstacleGrid[r][c] === 1)) {
                return 0;
        }

        if (memo[r][c] !== undefined) {
            return memo[r][c];
        }

        // if we reach the end
        if (r === numRows - 1 && c === numCols - 1) {
            return 1;
        }

        memo[r][c] = traverse(r + 1, c) + traverse(r, c + 1); 
        return memo[r][c];
    }

    return traverse(0, 0);
};


var uniquePathsWithObstacles = function(obstacleGrid) {
    const numRows = obstacleGrid.length;
    const numCols = obstacleGrid[0].length;

    const dp = Array.from({ length: numRows}, () => []);

    // dp[r][c] = # ways to get to obstacleGrid[r][c]

    // we know that there is 1 way to get to obstacleGrid[0][0]
    // UNLESS THE STARTING AREA IS AN OBSTACLE ITSELF
    dp[0][0] = obstacleGrid[0][0] === 1 ? 0 : 1;

    // fill out all rows
    for (let r = 1; r < numRows; r++) {
        // if dp[0][0] is an obstacle
        // there is no way to reach these later cells
        
        if (obstacleGrid[r][0] === 1) {
            dp[r][0] = 0;
        }
        else {
            dp[r][0] = dp[r - 1][0];
        }
    }

    // fill out all cols
    for (let c = 1; c < numCols; c++) {
        // if dp[0][0] is an obstacle
        // there is no way to reach these later cells
        
        if (obstacleGrid[0][c] === 1) {
            dp[0][c] = 0;
        }
        else {
            dp[0][c] = dp[0][c - 1];
        }
    }

    // since we require dp[r - 1][c] and dp[r][c - 1]
    // it is wise to loop in the opposite direction of your recurrence relation

    // since we've already solved all dp[0...numRows][0] and dp[0][0...numCols]
    // we can use these previous cells to solve newer cells
    for (let r = 1; r < numRows; r++) {
        for (let c = 1; c < numCols; c++) {
            if (obstacleGrid[r][c] === 1) {
                dp[r][c] = 0;
            }
            else {
                dp[r][c] = dp[r - 1][c] + dp[r][c - 1];
            }
        }
    }

    return dp[numRows - 1][numCols - 1];
}

## Minimum Path Sum

* https://leetcode.com/problems/minimum-path-sum/description/
***
* Time Complexity:
    - naive: O(2$^{m + n}$)
        * for every node, you have 2 options:
            - go down
            - go right
        * since, we only have these 2 options, if we start at [0, 0], we'll go all the way down first until we reach numRows, then we'll go all the way to the right until we reach [numRows - 1, numCols - 1]
            - this means the recursion will return when we go m + n times
    - topdown memo: O(m x n)
        * by memoizing, we only visit each cell once and there are m x n cells in grid
    - bottom-up dp: O(m x n)
        * we also visit every cell once
* Space Complexity:
    - naive: O(m + n)
        * uses recursion so requires space for functions in the stack
        * the number of functions in the stack is bounded by how deep the recursion goes
            - since we know that it will return once we go m + n times, the recursion is bounded by m + n functions
    - topdown memo: O(m x n)
        * requires space for the memo table
    - bottom-up dp: O(m x n)
        * requires space for the dp table
***
* naive:
    - pretty typical dp algorithm
    - we have 2 options, we either go down or go right
    - everytime we recurse, we keep track of a sum
    - when we go OoB, however, we return $\infty$, not the sum itself
        * reason being, if we just returned the original sum, Math.min would take that sum since as we traverse, our original sum gets bigger
        * by returning $\infty$, we make the algorithm disregard the OoB values
* topdown memo:
    - we have to change our algorithm here b/c the memo table actually doesn't do anything if we keep the algorithm the same
        * reason being, our naive algorithm keeps track of a sum as we move from [0,0] down to [m - 1, n - 1]
        * when we reach a cell we've seen and we return memo[r][c], that value represents the path from [0, 0] to [r, c] which is not what we want
        * we actually want the min path sum of [m - 1, n - 1] to [r, c]
        * therefore, when we reach grid[m - 1][n - 1] we actually return its value and not a sum
        * essentially we start counting back up when we reach grid[m - 1][n - 1] instead of counting along the way
        * we still keep OoB condition the same but we return memo[0][0]
* bottom-up dp:
    - looking at the grid, we notice that we can calculate the sums of the top row and left columns already since we know that dp[0][0] = grid[0][0]
        * we just need to create a dp table and fill those out
    - looking at the naive algorithm, we come up with this __recurrence relation__:
        - __dp[r][c] = grid[r][c] + Math.min(dp[r - 1][c] + dp[r][c - 1])__
    - now why did we do [r - 1] and [c - 1] instead of addition?
        * reason being, we can calculate those earlier values already and use them to calculate the next ones
    - similar to our naive algorithm, we calculate our sum as we go along from [0, 0] to [m - 1, n - 1]
    - since we've already calculated the top row and left column, we can just create 2 for loops that loop from:
        - r: 1...m - 1
        - c: 1 ... n - 1
    - then we return dp[m - 1][n - 1] b/c that is the minimum path sum from [0,0] ... [m - 1,n - 1]

In [1]:
/**
 * @param {number[][]} grid
 * @return {number}
 */

 // naive solution
var minPathSum = function(grid) {
    const numRows = grid.length;
    const numCols = grid[0].length;

    const traverse = (r, c, sum) => {
        // if we are OoB
        if ((r < 0 || r >= numRows) ||
            (c < 0 || c >= numCols)) {
                return Number.POSITIVE_INFINITY;
        }

        sum += grid[r][c];
        if ((r === numRows - 1) && (c === numCols - 1)) {
            return sum;
        }

        return Math.min(traverse(r + 1, c, sum), traverse(r, c + 1, sum));
    }

    return traverse(0, 0, 0);
};

// topdown memo
var minPathSum = function(grid) {
    const numRows = grid.length;
    const numCols = grid[0].length;
    if (numRows === 1 && numCols === 1) return grid[0][0];
    const memo = Array.from({ length: numRows}, () => []);

    const traverse = (r, c) => {
        // if we are OoB
        if ((r < 0 || r >= numRows) ||
            (c < 0 || c >= numCols)) {
                return Number.POSITIVE_INFINITY;
        }

        if ((r === numRows - 1) && (c === numCols - 1)) {
            return grid[r][c];
        }

        if (memo[r][c] !== undefined) {
            return memo[r][c];
        }

        memo[r][c] = grid[r][c] + Math.min(traverse(r + 1, c), traverse(r, c + 1));
        return memo[r][c];
    }

    traverse(0, 0);
    return memo[0][0];
};

// bottom-up dp
var minPathSum1 = function(grid) {
    const numRows = grid.length;
    const numCols = grid[0].length;
    const dp = Array.from({ length: numRows }, () => []);
    dp[0][0] = grid[0][0];

    // prefill top row
    for (let c = 1; c < numCols; c++) {
        dp[0][c] = grid[0][c] + dp[0][c - 1];
    }

    // prefill left col
    for (let r = 1; r < numRows; r++) {
        dp[r][0] = grid[r][0] + dp[r - 1][0];
    }

    for (let r = 1; r < numRows; r++) {
        for (let c = 1; c < numCols; c++) {
            dp[r][c] = grid[r][c] + Math.min(dp[r - 1][c], dp[r][c - 1]);
        }
    }

    return dp[numRows - 1][numCols - 1];
}